# MVP :

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
from PIL import Image
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.losses import categorical_crossentropy
import splitfolders
import shutil
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
import cv2,os
import random
from tensorflow.keras.optimizers import Adam 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
import plotly.graph_objects as go
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg19 import VGG19  
from tensorflow.keras.applications import mobilenet_v2
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score,f1_score, confusion_matrix
from keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout, Flatten,InputLayer
from tensorflow .keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score

## Read data images :

In [ ]:
data_path = 'images/'

In [ ]:
import os 
CATEGORIES = os.listdir(data_path)
print(CATEGORIES)
print(len(CATEGORIES))

In [ ]:
training_data = []
IMG_SIZE=128
def making_trian_dataset():
    for category in CATEGORIES:
        path = os.path.join(data_path, category) # path to our data
        class_num = CATEGORIES.index(category)# classifcation index
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resizing our imgz
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
making_trian_dataset()

In [ ]:
print(len(training_data))

## Shuffle data :

In [ ]:
random.shuffle(training_data)

## Data Labeled :

In [ ]:

X=[]
y=[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X)#.reshape(-1, IMG_SIZE,IMG_SIZE,3)
y = to_categorical(y, num_classes = len(CATEGORIES))

In [ ]:
X.shape,y.shape

## Data Splitting For Simple NN :

In [ ]:
nsample,nx,ny,ni=X.shape
X1=X.reshape(nsample,nx*ny*ni)
print(X1.shape)

In [ ]:
X1_train, X1_val_test, y1_train, y1_val_test = train_test_split(X1, y, test_size=.2, random_state=77)
X1_val, X1_test, y1_val, y1_test = train_test_split(X1_val_test, y1_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X1_train.shape},  labels: {y1_train.shape}")
print(f"Validation data: {X1_val.shape},  labels: {y1_val.shape}")
print(f"Testing data: {X1_test.shape},  labels: {y1_test.shape}")

## Data Splitting :

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=.2, random_state=77)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X_train.shape},  labels: {y_train.shape}")
print(f"Validation data: {X_val.shape},  labels: {y_val.shape}")
print(f"Testing data: {X_test.shape},  labels: {y_test.shape}")

## VGG19 Model :

In [ ]:
IMAGE_SIZE = [128, 128]

In [ ]:
model_vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)  

In [ ]:
# don't train existing weights
for layer in model_vgg19.layers:
  layer.trainable = False

In [ ]:
# useful for getting number of classes
folders = glob('try/*')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(model_vgg19.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model_20_vgg19 = Model(inputs=model_vgg19.input, outputs=prediction)

In [ ]:
# view the structure of the model
model_20_vgg19.summary()

In [ ]:
# tell the model what cost and optimization method to use

model_20_vgg19.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam', 
    metrics = ['accuracy'])

In [ ]:
model_20_vgg19.save_weights('weights_20_vgg19__11:30am.h5', save_format='h5')

In [ ]:
import tensorflow as tf
from tensorflow import keras

chechpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
result_V19 = model_20_vgg19.fit(X_train,y_train, 
                                validation_data=(X_val,y_val), 
                                batch_size=64,
                                epochs=20,
                                callbacks=[chechpoint]
                                )



In [ ]:
# loss
plt.plot(result_V19.history['loss'], label='train loss')
plt.plot(result_V19.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# accuracies
plt.plot(result_V19.history['accuracy'], label='train acc')
plt.plot(result_V19.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

## load model :

In [ ]:
import tensorflow as tf

from keras.models import load_model

model_vgg19.save('save_model_20_vgg19__11:30am.h5')

# Emotion detection :

In [ ]:
emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
face_image  = cv2.imread(r"/images/aidai/0_0_aidai_0014.jpg")
plt.imshow(face_image)
# resizing the image
face_image = cv2.resize(face_image, (48,48))
face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
model = load_model("model_v6_23.hdf5")
predicted_class = np.argmax(model.predict(face_image))
label_map = dict((v,k) for k,v in emotion_dict.items()) 
predicted_label = label_map[predicted_class]
print(predicted_label)